In [2]:
import keras
from keras.applications import VGG16
vgg_model = VGG16(weights='imagenet',
                               include_top=False,
                               input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 159s 3us/step


In [ ]:
# To see the models' architecture and layer names, run the following
vgg_model.summary()

In [4]:
from keras import optimizers
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
# Creating dictionary that maps layer names to the layers
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

# Getting output tensor of the last VGG layer that we want to include
x = layer_dict['block2_pool'].output

# Stacking a new simple convolutional network on top of it    
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(5, activation='softmax')(x)

# Creating new model. Please note that this is NOT a Sequential() model.
from keras.models import Model
custom_model = Model(input=vgg_model.input, output=x)

# Make sure that the pre-trained bottom layers are not trainable
for layer in custom_model.layers[:7]:
    layer.trainable = False

# Do not forget to compile it
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy'])

/home/teqip-cek/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [5]:
custom_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [7]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/home/teqip-cek/Desktop/chandhuP/Training',target_size = (150, 150),batch_size = 32,class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/home/teqip-cek/Desktop/chandhuP//Testing',target_size = (150, 150),batch_size = 32,class_mode = 'categorical')

Found 413 images belonging to 5 classes.
Found 103 images belonging to 5 classes.


In [13]:
from keras.callbacks import ModelCheckpoint
custom_model_json= custom_model.to_json()
with open("models/model.vgg","w") as json_file:
    json_file.write(custom_model_json)
print('starting training....')
checkpointer = ModelCheckpoint(filepath="models/vgg.hdf5", verbose=1, save_best_only=True)
history = custom_model.fit_generator(generator=training_set, steps_per_epoch=413//20,epochs=515//20,validation_data=test_set,validation_steps=102//20,callbacks=[checkpointer])
print('training finished!!')

starting training....
Epoch 1/25
20/20 [==============================] - 206s 10s/step - loss: 10.8677 - acc: 0.3185 - val_loss: 13.0139 - val_acc: 0.1926

Epoch 00001: val_loss improved from inf to 13.01387, saving model to models/vgg.hdf5
Epoch 2/25
20/20 [==============================] - 137s 7s/step - loss: 12.9679 - acc: 0.1940 - val_loss: 13.1333 - val_acc: 0.1852

Epoch 00002: val_loss did not improve from 13.01387
Epoch 3/25
20/20 [==============================] - 128s 6s/step - loss: 13.3298 - acc: 0.1730 - val_loss: 13.1333 - val_acc: 0.1852

Epoch 00003: val_loss did not improve from 13.01387
Epoch 4/25
20/20 [==============================] - 132s 7s/step - loss: 13.1305 - acc: 0.1854 - val_loss: 13.3341 - val_acc: 0.1727

Epoch 00004: val_loss did not improve from 13.01387
Epoch 5/25
20/20 [==============================] - 122s 6s/step - loss: 13.3802 - acc: 0.1699 - val_loss: 13.1333 - val_acc: 0.1852

Epoch 00005: val_loss did not improve from 13.01387
Epoch 6/25
20/